In [1]:
!pip install pandas-datareader

In [1]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

In [2]:
class AI_Trader():
  
  def __init__(self, state_size, action_space=3, model_name="AITrader"): #Stay, Buy, Sell
    
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name
    
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    
    self.model = self.model_builder()
    
  def model_builder(self):
    
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
    
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
    
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))
    
    return model
  
  def trade(self, state):
    
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    actions = self.model.predict(state)
    return np.argmax(actions[0])
  
  
  def batch_train(self, batch_size):
    
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
      
    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

In [3]:
dataset = data_reader.DataReader('AAPL', data_source="yahoo")
dataset[0:5]

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2015-02-05,120.230003,119.250000,120.019997,119.940002,42246200.0,110.637886
2015-02-06,120.250000,118.449997,120.019997,118.930000,43706600.0,109.706200
2015-02-09,119.839996,118.430000,118.550003,119.720001,38889800.0,110.434944
2015-02-10,122.150002,120.160004,120.169998,122.019997,62008500.0,112.556557
2015-02-11,124.919998,122.500000,122.769997,124.879997,73561800.0,115.194748


In [4]:
close = dataset['Close']
close[0:5]

Date
2015-02-05    119.940002
2015-02-06    118.930000
2015-02-09    119.720001
2015-02-10    122.019997
2015-02-11    124.879997
Name: Close, dtype: float64

In [5]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

def dataset_loader(stock_name):
  
  #Complete the dataset loader function
  dataset = data_reader.DataReader(stock_name, data_source="yahoo")
  
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  
  close = dataset['Close']

  return close

def state_creator(data, timestep, window_size):
  
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])

In [6]:
rr = state_creator(close, 20, 11)

In [7]:
print(rr)

[[0.97068777 0.30364468 0.03292624 0.83617031 0.12346797 0.65248711
  0.56709395 0.30576211 0.10621598 0.54735633]]


In [8]:
c = 0
for t in tqdm(range(10000000)):
    c += 4

100%|█████████████████████████████████████████████████████████████████| 10000000/10000000 [00:04<00:00, 2225051.29it/s]


In [9]:
stock_name = "AAPL"
data = dataset_loader(stock_name)

In [10]:
window_size = 10
episodes = 1000

batch_size = 32
data_samples = len(data) - 1

In [11]:
trader = AI_Trader(window_size)

In [12]:
trader.model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


In [13]:
for episode in range(1, episodes + 1):
  
  print("Episode: {}/{}".format(episode, episodes))
  
  state = state_creator(data, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory = []
  
  for t in tqdm(range(data_samples)):
    
    action = trader.trade(state)
    
    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0
    
    if action == 1: #Buying
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stocks_price_format(data[t]))
      
    elif action == 2 and len(trader.inventory) > 0: #Selling
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")
    
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
      
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))
    

Episode: 1/1000


  0%|                                                                                         | 0/1257 [00:00<?, ?it/s]

AI Trader bought:  $ 119.940002
AI Trader bought:  $ 118.930000
AI Trader sold:  $ 119.720001  Profit: - $ 0.220001
AI Trader bought:  $ 122.019997
AI Trader sold:  $ 126.459999  Profit: $ 7.529999
AI Trader sold:  $ 127.830002  Profit: $ 5.810005
AI Trader bought:  $ 133.000000
AI Trader sold:  $ 132.169998  Profit: - $ 0.830002
AI Trader bought:  $ 129.089996
AI Trader sold:  $ 129.360001  Profit: $ 0.270004
AI Trader bought:  $ 127.139999
AI Trader bought:  $ 124.949997
AI Trader sold:  $ 128.470001  Profit: $ 1.330002
AI Trader sold:  $ 127.500000  Profit: $ 2.550003


  2%|█▉                                                                             | 30/1257 [00:00<00:04, 294.91it/s]

AI Trader bought:  $ 125.900002
AI Trader bought:  $ 123.379997
AI Trader sold:  $ 124.239998  Profit: - $ 1.660004
AI Trader sold:  $ 123.250000  Profit: - $ 0.129997
AI Trader bought:  $ 126.370003


...

AI Trader sold:  $ 96.910004  Profit: $ 0.810005


 21%|████████████████▊                                                              | 267/1257 [12:53<46:22,  2.81s/it]

AI Trader bought:  $ 96.690002


 21%|████████████████▉                                                              | 270/1257 [13:01<43:52,  2.67s/it]

KeyboardInterrupt: 

In [1]:
%%html
<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 450 400" width="200" height="200">
  <rect x="80" y="60" width="250" height="250" rx="20" style="fill:red; stroke:black; fill-opacity:0.7" />
  <rect x="180" y="110" width="250" height="250" rx="40" style="fill:blue; stroke:black; fill-opacity:0.5;" />
</svg>

In [2]:
%%html
<input>Text:</input>

In [3]:
import tensorflow as tf 

if tf.test.gpu_device_name(): 

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:

   print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0
